## Imports

In [41]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader, random_split
from tqdm.notebook import tqdm_notebook

import os

from model.model import ModelV1
from utils.dataset import ReturnsDataset

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Global variables

In [42]:
BATCH_SIZE = 128
VAL_RATIO = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Get dataset splits and loaders

In [70]:
dataset = ReturnsDataset(split='train')
train_data, val_data = random_split(dataset=dataset, lengths=[(1-VAL_RATIO), VAL_RATIO])

test_data = ReturnsDataset(split='test')

In [71]:
train_loader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

val_loader = DataLoader(
    dataset=val_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

## Average meter class to measure avg loss and accuracy

In [45]:
import math

class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / (self.count + 1e-8)

## Define accuracy metric

In [46]:
def get_accuracy(y_true, y_pred):
    scores = []
    for true, pred in zip(y_true, y_pred):
        scores.append(true == pred)
    avg_score = np.mean(scores)
    return avg_score

## Train and validation loops

In [47]:
def train_loop(data_loader: torch.utils.data.DataLoader,
               model: nn.Module, 
               optimizer: torch.optim.Optimizer,
               criterion: nn.Module,
               epoch: int):
    model.train()
    loss_avg = AverageMeter()
    
    for x, targets in tqdm_notebook(data_loader, desc='Training progress'):
        model.zero_grad()
        x = x.to(torch.float32)
        targets = targets.to(torch.float32)
        
        preds = model(x)
        # probs = torch.softmax(preds, dim=1)

        loss = criterion(preds, targets.long())
        loss_avg.update(val=loss.item(), n=len(targets))
        loss.retain_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch}, Loss: {loss_avg.avg} ')

    return loss_avg.avg


In [48]:
def val_loop(data_loader: torch.utils.data.DataLoader, 
             model: nn.Module,
             device: torch.device,
             batch_size: int):
    
    acc_avg = AverageMeter()
    
    for x, target in tqdm_notebook(data_loader, desc='Validation progress'):
        probs = predict(x, model=model, device=DEVICE)
        acc_avg.update(val=get_accuracy(target, torch.argmax(probs, dim=1)), n=batch_size)
            
    
    print(f'Validation accuracy: {acc_avg.avg}')

    return acc_avg.avg

## Predict function

In [49]:
def predict(X, model, device):
    model.eval()
    X = X.to(device)
    X = X.to(torch.float32)
    
    with torch.inference_mode():
        logits = model(X)
        
    return nn.functional.softmax(logits, dim=1)

In [25]:
model = ModelV1(
    in_shape=test_data.shape[1],
    out_shape=5,
    hidden_units=10
)

optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=0.00001,
    weight_decay=0.01
)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, 
    mode='min', 
    factor=0.1,
    patience=10
)

epochs = tqdm_notebook(range(10), desc='Epochs')

best_acc = -np.inf

for epoch in epochs:
    loss_avg = train_loop(
        data_loader=train_loader,
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        epoch=epoch
    )

    acc_avg = val_loop(
        data_loader=val_loader,
        model=model,
        device=DEVICE,
        batch_size=BATCH_SIZE
    )
    if acc_avg > best_acc:
            best_acc = acc_avg
            model_save_path = os.path.join('model/', f'model-{epoch}-{acc_avg:.4f}.pth')
            torch.save(model.state_dict(), model_save_path)
            print('Model weights have been saved')

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Training progress:   0%|          | 0/1984 [00:00<?, ?it/s]

Epoch: 0, Loss: 340.6641918514498 


Validation progress:   0%|          | 0/221 [00:00<?, ?it/s]

Validation accuracy: 0.48291252932779977
Model weights have been saved


...

Validation accuracy: 0.4839324618734673
Model weights have been saved


Training progress:   0%|          | 0/1984 [00:00<?, ?it/s]

Epoch: 9, Loss: 1.3832574091188372 


Validation progress:   0%|          | 0/221 [00:00<?, ?it/s]

Validation accuracy: 0.4615921421985966


## Seems like custom made classifier is not doing so well
## Let's try One vs. Rest Logistic Regression for multiclass classification

In [62]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

In [65]:
# rf_classifier = RandomForestClassifier(
#     warm_start=True, 
#     n_estimators=1, 
#     criterion='entropy',
#     random_state=42,
#     # n_jobs=-1
# )

model = SGDClassifier(loss='log_loss')

In [72]:
classes = range(5)

epochs = tqdm_notebook(range(10), desc='Epochs')

for epoch in epochs:
    acc_avg = AverageMeter()
    
    for X, targets in tqdm_notebook(train_loader, desc='Training progress'):
        # X = X.numpy()
        # targets = targets.numpy()
        
        model.partial_fit(X, targets, classes=classes)
        # rf_classifier.n_estimators += 1

    for X, targets in tqdm_notebook(val_loader, desc='Validation progress'):
        # X = X.numpy()
        # targets = targets.numpy()
        
        preds = model.predict(X)
        acc_avg.update(val=get_accuracy(targets, preds), n=BATCH_SIZE)

    print(f'Epoch: {epoch}, Accuracy: {acc_avg.avg:4f}')

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Training progress:   0%|          | 0/1984 [00:00<?, ?it/s]

Validation progress:   0%|          | 0/221 [00:00<?, ?it/s]

Epoch: 0, Accuracy: 0.265808


Training progress:   0%|          | 0/1984 [00:00<?, ?it/s]

...

Validation progress:   0%|          | 0/221 [00:00<?, ?it/s]

Epoch: 8, Accuracy: 0.486356


Training progress:   0%|          | 0/1984 [00:00<?, ?it/s]

Validation progress:   0%|          | 0/221 [00:00<?, ?it/s]

Epoch: 9, Accuracy: 0.486404


## Conclusion and ways to improve

### Model is not the decisive factor of strong predictions here. 
 To further improve we need to work on features. The most significant way to improve would be to create multilingual embeddings based on reviews. We either use M-BERT(train ourselves to russian, cyrillic uzbek and latin uzbek) or TF-IDF(stop words for russian, cyrillic uzbek and latin uzbek). Moreover, we need to add separation for reviews based on language.